In [1]:
import numpy as np
from numpy import linalg as LA
import pandas as pd
from math import radians, cos, sin, asin, sqrt, isnan
import random
from ortools.sat.python import cp_model
import time

In [2]:
#-----------------------------------EXTRACT THE DATA-----------------------------------
#start_time = time.time()

# users-melbcbd-generated.csv contains:
# •  Latitude-Longitude
# of the users in the Melbourne CBD area.
requests_path = '..\\eua-dataset\\users\\'
R = pd.read_csv(requests_path + 'users-test.csv') # test with 8 users
#R = pd.read_csv(requests_path + 'users-test-more-users.csv') # test with 14 users
#R = pd.read_csv(requests_path + 'users-melbcbd-generated.csv') # test with 816 users

# site-optus-melbCBD.csv contains:
# •  SiteID-Latitude-Longitude-Name-State-LicensingAreaID-PostCode-SitePrecision-Elevation-HCISL2
# of all Optus BS in Melbourne CBD area (edge-servers)
nodes_path = '..\\eua-dataset\\edge-servers\\'
#N = pd.read_csv(nodes_path + 'serverstest.csv') # test with 3 servers
N_src = pd.read_csv(nodes_path + 'serverstest.csv') # test with 3 servers
N_dest = pd.read_csv(nodes_path + 'serverstest.csv') # test with 3 servers
#N = pd.read_csv(nodes_path + 'serverstest-more-servers.csv') # test with 6 servers
#N = pd.read_csv(nodes_path + 'site-optus-melbCBD.csv') # test with 125 servers

In [3]:
#-----------------------------------INPUTS-----------------------------------

#Incoming requests to node i
lambda_ri = [[1,1,1,1],[1],[1,1,1]]

# Memory of function: m_f
m1 = 1
m2 = 2
m3 = 3
m4=4
m_f = [m1,m2,m3,m4]


""" 
m_request = np.empty((len(N_src),len(R)))
for r in range(len(R)):
    for i in range(len(N_src)):
      m_request[i][r]=random.choice(m_f) """

""" m_temp = np.resize(m_f,len(R))
for r in range(len(R)):
  for i in range (len(N_src)):
        m_request[i][r]=m_temp[r] """

""" m_request=[]
for i in range (len(N_src)):
      m_request.append(np.random.choice(m_f,len(lambda_ri[i]))) """

#matrix that assignes a function to each request [N x l]
m_request=[]
for i in range (len(N_src)):
      m_request.append(np.resize(m_f,len(lambda_ri[i]))) 
    
# Sort the requests by their memory requirement [N x l] --- returns position of the [] where request is found
index= []
for i in range (len(N_src)):
  index.append(sorted(range(len(m_request[i])), key=lambda a: m_request[i][a]))

#shows which requests are assigned to each function [F x R]
req_distr=np.zeros([len(m_f),len(R)])

r=0
while r<len(R):
  for i in range(len(N_src)):
    for l in range(len(lambda_ri[i])):
      for f in range (len(m_f)):
        if m_request[i][l]==m_f[f]:
          req_distr[f][r]=1
          r=r+1
  
#Maximum allowed network delay for function f:  phi_f
phi_f=[]

In [4]:
final_memory=np.zeros([len(m_f),len(R)])

r=0
while r<len(R):
  for i in range(len(N_src)):
    for l in range(len(lambda_ri[i])):
      for f in range (len(m_f)):
        if m_request[i][l]==m_f[f]:
          final_memory[f][r]=m_f[f]
          r=r+1

In [5]:
#-----------------------------------INFRASTRUCTURED DATA-----------------------------------

#Memory available in node j: M_j
M_j = [2,17,17]

#Cores on node j: U_j
U_j=[25,26,27]

In [6]:
#-----------------------------------MONITORED DATA-----------------------------------
#Network delay between nodes i and j
delta_ij = []

#Cores used by node j per single request: u_rj (???????????????????????????? Per function or request)
u_rj = [[2,2,2,2,2,2,2,2],[2,2,2,2,2,2,2,2],[2,2,2,2,2,2,2,2]]

In [7]:
#-----------------------------------VARIABLES-----------------------------------

U_ni = [] # Set of requests conected to node n

#establish which containers (function instances) are allocated to each node (???????????? is it given or calculated by variable c[f,j])
container_loc = [[1,0,1,0],[0,1,0,1],[1,0,0,1]]

#Gives the index of the requests for each function (necesary????????????????)
F_r=[]
for row in range(len(req_distr)):
    temp_F=[]
    for column in range(len(R)):
        if req_distr[row][column]==1:
            temp_F.append(column)
    F_r.append(temp_F)

#Creates vector with the indexes of all requests
final_index =[]
for f in range(len(F_r)):
  temp=F_r[f]
  for p in range(len(temp)):
    final_index.append(temp[p])

#Represents nodes each request can be allocated to [N x R]
posible_n=np.zeros([len(N_dest),len(R)])
for j in range(len(N_dest)):
  for r in final_index:
    for f in range (len(m_f)):
      if req_distr[f][r]==1 and container_loc[j][f]==1:
        posible_n[j][r]=1

#1 if request r arrives to node i [N x R]
loc_arrival_r=np.zeros([len(N_src),len(R)])
r=0
while r<len(R):
    for i in range(len(N_src)):
        for l in range(len(lambda_ri[i])):
            loc_arrival_r[i][r]=1
            r=r+1

In [8]:
#-----------------------------------HAVERSINE-----------------------------------

def haversine(lon1, lat1, lon2, lat2):
  
    # Convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [9]:
#-----------------------------------COVERAGE REQUEST-NODE-----------------------------------

#radius = np.round(np.random.uniform(0.1,0.15,len(S)),3) # in km
radius = np.full(len(N_src), 0.03)

for i in range(len(N_src)):
  node_latitude = N_src.iloc[i]['LATITUDE']
  node_longitude = N_src.iloc[i]['LONGITUDE']
  temp = []
  for r in range(len(R)):
    request_latitude = R.iloc[r]['Latitude']
    request_longitude = R.iloc[r]['Longitude']

    dist_geo = haversine(node_longitude, node_latitude, request_longitude, request_latitude)
    
    if dist_geo <= radius[i]:
        temp.append(1)
        
    else:
        temp.append(0)
       
  U_ni.append(temp)


In [10]:
#---------------------------------DISTANCE BETWEEN NODES---------------------------------

#radius = np.round(np.random.uniform(0.1,0.15,len(S)),3) # in km
radius = np.full(len(N_src), 0.03)

for i in range(len(N_src)):
  node1_latitude = N_src.iloc[i]['LATITUDE']
  node1_longitude = N_src.iloc[i]['LONGITUDE']
  temp_dist = []
  for j in range(len(N_dest)):
    node2_latitude = N_dest.iloc[j]['LATITUDE']
    node2_longitude = N_dest.iloc[j]['LONGITUDE']

    dist_geo_nodes = haversine(node1_longitude, node1_latitude, node2_longitude,node2_latitude)
    #print(dist_geo)
    #print(radius[i]+radius[j])
    temp_dist.append(np.round(dist_geo_nodes,3))
       
  delta_ij.append(temp_dist)

for r1 in range(len(N_src)):
  temp_delay = []
  for r2 in range(len(N_dest)):
    if r1==r2:
      temp_delay.append(0)
    else:
      temp_delay.append(radius[r1]+radius[r2])
  phi_f.append(temp_delay)
    

In [11]:
#---------------------------------- CP MODEL ----------------------------------
model = cp_model.CpModel()

#-----------------------------------SOLVER VARIABLES-----------------------------------

# x_j,r = True if request r is allocated to node j
x = {}
for j in range(len(N_dest)):
    for r in final_index:
        x[j, r] = model.NewBoolVar(f'c[{j}][{r}]')

# c_f,j = True if container f (fa,fb,fc) is deployed on node j
c = {}
for f in range(len(F_r)):
    for j in range(len(N_dest)):
        c[f, j] = model.NewBoolVar(f'c[{f}][{j}]')

# y_j = True if node j is used
# y_j = False otherwise
y = {}
for j in range(len(N_dest)): 
    y[j] = model.NewBoolVar(f'c[{j}]')
 
print(model.ModelStats())

satisfaction model '':
#Variables: 39
  - 39 Booleans in [0,1]



In [12]:
""" for j in range(len(N_dest)):
    for r in final_index:
        if loc_arrival_r[j][r]==1 and posible_n[j][r]==0:
            model.Add(x[j, r]==0) 
            
        elif loc_arrival_r[j][r]==0 and posible_n[j][r]==0:
            model.Add(x[j, r]==0) """
             

' for j in range(len(N_dest)):\n    for r in final_index:\n        if loc_arrival_r[j][r]==1 and posible_n[j][r]==0:\n            model.Add(x[j, r]==0) \n            \n        elif loc_arrival_r[j][r]==0 and posible_n[j][r]==0:\n            model.Add(x[j, r]==0) '

In [13]:
distFin =[]
ind_min=[]

for r in range(len(R)):
    for i in range(len(N_src)):
            dist_temp=[]
            ind_min_temp=[]
            for j in range(len(N_dest)):
                if posible_n[j][r]==1 and loc_arrival_r[i][r]==1:
                    dist_temp.append(delta_ij[i][j])
                    ind_min_temp.append(j)
            
            if dist_temp!=[]:
                distFin.append(dist_temp)
                ind_min.append(ind_min_temp)

In [14]:
"""  
    value=distFin[r].index(min(distFin[r]))
    indice=ind_min[r][value]
    model.Add(x[indice, r]==1)"""

'  \n    value=distFin[r].index(min(distFin[r]))\n    indice=ind_min[r][value]\n    model.Add(x[indice, r]==1)'

In [15]:
#-----------------------------------CONSTRAINTS-----------------------------------

#Controls if request r can be managed by node j

for j in range(len(N_dest)):
    for r in final_index:
        if posible_n[j][r]==0:
            model.Add(x[j, r]==0)  
        

#Proximity constraint (node i-node j) 
for i in range(len(N_src)):
    for r in final_index:
        for j in range(len(N_dest)):
                if delta_ij[i][j]> phi_f[i][j]:
                    model.Add(
                    x[j, r]==0
                    )  


#1 if container of function f is allocated to node j
""" for j in range(len(N_dest)):
    for f in range(len(F_r)):
        if container_loc[j][f]==0:
            model.Add(c[f,j]==0)  """
       

#Container deployment: True if container of function f is deployed on node j
for f in range(len(F_r)):
    for j in range(len(N_dest)):
        model.Add(sum([x[j,r] for r in final_index])<= c[f,j]*1000)
    

# Capacity constraint (memory)
for j in range(len(N_dest)):
    for f in range(len(F_r)):
        model.Add(
                sum([
                    int(final_memory[f][r]) * x[j,r]*req_distr[f][r] for r in final_index
                    ]) <= M_j[j]
                )   
    
""" for j in range(len(N_dest)):
    for l in range(len(m_request[j])):
        model.Add(
                sum([
                    m_request[j][l] * c[f,j] for f in range(len(F_r))
                    ]) <= M_j[j]
                )   """


"""
# Avoid resource contention
for j in range(len(N_dest)):
        model.Add(
                sum([sum([x[j,r]*u_rj[j][r] for r in final_index]) for j in range(len(N_dest))
                ]) <= U_j[j]*y[j]
            )   
"""
# Contraint family (each request can be allocated just once)

for r in final_index:
    model.Add(sum([x[j, r] for j in range(len(N_dest))]) <= 1)

 
print(model.ModelStats())

satisfaction model '':
#Variables: 39
  - 39 Booleans in [0,1]
#kLinear1: 15
#kLinear2: 6
#kLinear3: 11
#kLinearN: 12 (#terms: 108)


In [16]:
#---------------------------------- CP SOLVER ----------------------------------
solver = cp_model.CpSolver()

#---------------------------------- OBJECTIVE FUNCTIONS ---------------------------------

# Maximize the number of allocated requests
objective_max = []

for j in range(len(N_dest)):
    for r in final_index:
        objective_max.append(x[j, r])
model.Maximize(sum(objective_max))

solver.Solve(model)
max_requests = solver.ObjectiveValue()

# Hint (speed up solving)
for j in range(len(N_dest)):
     for r in final_index:
        model.AddHint(x[j,r], solver.Value(x[j,r]))
        
for j in range(len(N_dest)):
    model.AddHint(y[j], solver.Value(y[j]))

""" for f in range(len(F_r)):
    for j in range(len(N_dest)):
        model.AddHint(c[f,j],solver.Value(c[f,j])) """
        
# Constraint previous objective
model.Add(
    sum([
        x[j, r] for j in range(len(N_dest)) for r in final_index
    ]) == round(solver.ObjectiveValue())
)

# Minimize the number of nodes used
objective_min = []
for j in range(len(N_dest)):
        objective_min.append(y[j])
model.Minimize(sum(objective_min))

print(model.ModelStats())

optimization model '':
#Variables: 39 (#bools:3 in objective)
  - 39 Booleans in [0,1]
#kLinear1: 15
#kLinear2: 6
#kLinear3: 11
#kLinearN: 13 (#terms: 132)


In [17]:
#-----------------------------------CALL THE SOLVER-----------------------------------
status = solver.Solve(model)

#-----------------------------------DISPLAY THE SOLUTION-----------------------------------

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('SOLUTION:')
    print(f'Objective value: {max_requests} requests have been allocated to {solver.ObjectiveValue()} nodes\n')
    for r in final_index:
        for j in range(len(N_dest)):
            if int(solver.Value(x[j,r])) == 1:
                print(f'x[{j},{r}]: Request {r} has been allocated to node {j}')
    print('----------------------------------------------------------------------')
    for f in range(len(F_r)):
        for j in range(len(N_dest)):
            if int(solver.Value(c[f,j])) == 1:
                print(f'c[{f},{j}]: Function {f} has been deployed on node {j}')

    print('----------------------------------------------------------------------')
    for j in range(len(N_dest)):
        if int(solver.Value(y[j])) == 1:
                print(f'y[{j}]: Node {j} is used')
    #---------------------------------------------CERO VARIABLES----------------------------------------------------
    print('  ')
    print('----------------------------------------------------------------------')
    print('VARIABLES IN CERO')
    print('----------------------------------------------------------------------')
    for r in final_index:
        for j in range(len(N_dest)):
            if int(solver.Value(x[j,r])) == 0:
                print(f'x[{j},{r}]')
    print('----------------------------------------------------------------------')
    for f in range(len(F_r)):
        for j in range(len(N_dest)):
            if int(solver.Value(c[f,j])) == 0:
                print(f'c[{f},{j}]')

    print('----------------------------------------------------------------------')
    for j in range(len(N_dest)):
        if int(solver.Value(y[j])) == 0:
                print(f'y[{j}]')
#else:
 #   print('The problem does not have an optimal solution.')

#print("\n--- Run time: %s seconds ---" % round((time.time() - start_time),2))

SOLUTION:
Objective value: 6.0 requests have been allocated to 0.0 nodes

x[2,0]: Request 0 has been allocated to node 2
x[2,4]: Request 4 has been allocated to node 2
x[2,5]: Request 5 has been allocated to node 2
x[1,1]: Request 1 has been allocated to node 1
x[1,6]: Request 6 has been allocated to node 1
x[2,3]: Request 3 has been allocated to node 2
----------------------------------------------------------------------
c[0,1]: Function 0 has been deployed on node 1
c[0,2]: Function 0 has been deployed on node 2
c[1,1]: Function 1 has been deployed on node 1
c[1,2]: Function 1 has been deployed on node 2
c[2,1]: Function 2 has been deployed on node 1
c[2,2]: Function 2 has been deployed on node 2
c[3,1]: Function 3 has been deployed on node 1
c[3,2]: Function 3 has been deployed on node 2
----------------------------------------------------------------------
  
----------------------------------------------------------------------
VARIABLES IN CERO
----------------------------------